# ULTIMO tutorial 5: Finalize attributes for trip generation and distribution

In this tutorial, the attraction factors per TAZ that influence trip generation and distribution will be determined using the `AttractionFactors` methods.

The inputs are:
- TAZ as GeoDataFrame
- Population density as point GeoDataFrame

In [1]:
# for network creation
from ultimo import AttractionFactors

In [2]:
# for reading and saving files etc.
import geopandas as gpd
import pandas as pd
import numpy as np
# for time tracking
from datetime import datetime

## Read input: TAZ and population

The input includes georeferenced TAZ in `EPSG:4326` with the following _required_ attributes (columns):

* __ID__ | field including a unique ID, e.g. the NUTS ID
* __Country__ | field containing the ISO-2 code of the respective country

Other attributes like name etc. can be added, but are not required for the following steps.

To determine population per TAZ, a GeoDataFrame containing points with population on a 1km grid is used, based on [GHSL](https://ghsl.jrc.ec.europa.eu/download.php?ds=pop). 

In [3]:
# load taz from database or local
taz = gpd.GeoDataFrame.from_file('tutorial-files/_input/taz-tutorial.gpkg')

# defining ID, country and geometry column names of taz
taz_id = "nuts_id"
taz_cn = "cntr_code"
taz_geo = "geometry"

taz.head()

,nuts_id,cntr_code,nuts_name,geometry
0,FI193,FI,Keski-Suomi,"MULTIPOLYGON (((26.13865 63.45759, 26.16055 63..."
1,FI194,FI,Etelä-Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.63880 62..."
2,FI195,FI,Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.52578 62..."
3,FI196,FI,Satakunta,"MULTIPOLYGON (((21.41993 61.04744, 21.42015 61..."
4,FI197,FI,Pirkanmaa,"MULTIPOLYGON (((22.83124 62.27089, 22.90118 62..."


In [4]:
countries = taz[taz_cn].unique()

In [5]:
# read population
pop = gpd.GeoDataFrame.from_file('tutorial-files/_input/4326-points-population.gpkg')

## Attraction factors on cell level

For the attraction factors on cell level, which will be used for trip generation and distribution, __population__ and __industry__ data is aggregated per TAZ. This information will then be transformed to an attraction index for

- national distribution
- international distribution

In [6]:
path_export = 'tutorial-files/tutorial5/'

In [7]:
att = AttractionFactors.AttractionIndex(taz)

### Population

Get population per TAZ based on population density from a `Point GeoDataFrame`

In [8]:
# population
print("... get population {}".format(datetime.now()))
att.population_from_point(pop)
print("finished at {}".format(datetime.now()))

... get population 2023-03-16 10:50:22.500211
finished at 2023-03-16 10:59:48.201868


### Industry

Get industry areas from OSM and aggreate total industrial area, number of industrial areas per TAZ

In [9]:
# industry
print("... get industrial sites from OSM {}".format(datetime.now()))
att.industry_attributes_from_osm()
print("finished at {}".format(datetime.now()))

... get industrial sites from OSM 2023-03-16 10:59:48.211931


69it [23:04, 20.06s/it]

finished at 2023-03-16 11:22:52.230783


### Calculate attraction index

The attraction index is calculated per TAZ using the values for population, industrial site count and aggregated area in relation to the mean. This means, the index for national transport is different to the international index. Hence, we set the scope in the `att.attraction_index()` function for the national index and leave is as `None` for the international index.

For each country (national)

In [14]:
# iterate over countries, calculate index per country and concat TAZ per country to final TAZ GeoDataFrame
for i, cn in enumerate(countries):
    taz_cn = att.attraction_index(scope=cn)
    if i == 0:
        taz_all_nat = taz_cn.copy()
    else:
        taz_all_nat = pd.concat([taz_all_nat, taz_cn])
taz_all_nat.head()

,nuts_id,cntr_code,nuts_name,geometry,population,ind_area_count,ind_area_sum,index_nat
0,FI193,FI,Keski-Suomi,"MULTIPOLYGON (((26.13865 63.45759, 26.16055 63...",510850.572340,282,2.246628e+07,0.903992
1,FI194,FI,Etelä-Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.63880 62...",358466.021589,396,2.183331e+07,0.818465
2,FI195,FI,Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.52578 62...",340041.957401,276,1.842693e+07,0.698118
3,FI196,FI,Satakunta,"MULTIPOLYGON (((21.41993 61.04744, 21.42015 61...",358255.966908,390,3.533939e+07,0.919390
4,FI197,FI,Pirkanmaa,"MULTIPOLYGON (((22.83124 62.27089, 22.90118 62...",901022.640876,505,4.031231e+07,1.607394


International

In [15]:
# get international index
taz_int = att.attraction_index()
taz_int.head()

,nuts_id,cntr_code,nuts_name,geometry,population,ind_area_count,ind_area_sum,index_int
0,FI193,FI,Keski-Suomi,"MULTIPOLYGON (((26.13865 63.45759, 26.16055 63...",510850.572340,282,2.246628e+07,0.889445
1,FI194,FI,Etelä-Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.63880 62...",358466.021589,396,2.183331e+07,0.805294
2,FI195,FI,Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.52578 62...",340041.957401,276,1.842693e+07,0.686884
3,FI196,FI,Satakunta,"MULTIPOLYGON (((21.41993 61.04744, 21.42015 61...",358255.966908,390,3.533939e+07,0.904596
4,FI197,FI,Pirkanmaa,"MULTIPOLYGON (((22.83124 62.27089, 22.90118 62...",901022.640876,505,4.031231e+07,1.581528


Merge both national and intenational index to final TAZ `GeoDataFrame` with attraction factors

In [16]:
taz_final = taz_all_nat.merge(taz_int[[taz_id, 'index_int']], on=taz_id, how='left')
taz_final.head()

,nuts_id,cntr_code,nuts_name,geometry,population,ind_area_count,ind_area_sum,index_nat,index_int
0,FI193,FI,Keski-Suomi,"MULTIPOLYGON (((26.13865 63.45759, 26.16055 63...",510850.572340,282,2.246628e+07,0.903992,0.889445
1,FI194,FI,Etelä-Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.63880 62...",358466.021589,396,2.183331e+07,0.818465,0.805294
2,FI195,FI,Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.52578 62...",340041.957401,276,1.842693e+07,0.698118,0.686884
3,FI196,FI,Satakunta,"MULTIPOLYGON (((21.41993 61.04744, 21.42015 61...",358255.966908,390,3.533939e+07,0.919390,0.904596
4,FI197,FI,Pirkanmaa,"MULTIPOLYGON (((22.83124 62.27089, 22.90118 62...",901022.640876,505,4.031231e+07,1.607394,1.581528


In [17]:
taz_final.to_file(path_export + 'taz-tutorial-attindex' + '.gpkg', driver='GPKG')